# CNN - ConvNextV2

https://huggingface.co/facebook/convnextv2-tiny-1k-224

In [1]:
import pandas as pd
from transformers import pipeline

classifier = pipeline("image-classification", model="facebook/convnextv2-tiny-1k-224")

t:\win_programs\python_venvs\gnn-homeworks\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from PIL import Image
import os
from tqdm import tqdm
import shutil

    
def classify_images(folder_name, output_suffix, drive_path_root):

    path_to_images = drive_path_root + '/' + folder_name

    image_paths = [path_to_images +'/'+p for p in os.listdir(path_to_images) if p.endswith(".png")]
    print(f'Found {len(image_paths)} images in {path_to_images}')

    model_predictions_top_5 = []
    model_predictions_top_1 = []

    for image_path in tqdm(image_paths):
        image = Image.open(image_path).convert("RGB")
        prediction = classifier(image)
        model_predictions_top_5.append({
            "image_path": image_path,
            "predictions": prediction
        })
        model_predictions_top_1.append({
            "image_path": image_path,
            "prediction": prediction[0]['label'],
            "score": round(prediction[0]['score'], 2)
        })

    output_top_1_name = f"classifier_predictions_top_1_{output_suffix}.csv"
    output_top_5_name = f"classifier_predictions_top_5_{output_suffix}.csv"

    # save model_predictions_top_1 as a csv
    df = pd.DataFrame(model_predictions_top_1)
    df.to_csv(output_top_1_name, index=True)

    # save model_predictions_top_5 as a csv
    # model_predictions_top_5 has a nested list of dictionaries, so we need to flatten it
    model_predictions_all_flat = []
    for image in model_predictions_top_5:
        for prediction_position, prediction in enumerate(image['predictions']):
            model_predictions_all_flat.append({
                "image_path": image['image_path'],
                "prediction_position": prediction_position,
                "label": prediction['label'],
                "score": round(prediction['score'], 2)
            })

    # save model_predictions_all_flat as a csv
    df = pd.DataFrame(model_predictions_all_flat)
    df.to_csv(output_top_5_name, index=True)

    # copy to google drive
    shutil.copyfile(output_top_1_name, drive_path_root + '/' + output_top_1_name)
    print("Finished, saved to", drive_path_root + '/' + output_top_1_name)

    shutil.copyfile(output_top_5_name, drive_path_root + '/' + output_top_5_name)
    print("Finished, saved to", drive_path_root + '/' + output_top_5_name)

    # clean up
    os.remove(output_top_1_name)
    os.remove(output_top_5_name)

In [5]:
root_path = "datasets_generated_ready_models/stylegan2"
folder_name = "generated_images"
output_suffix = 'cars'

classify_images(folder_name, output_suffix, drive_path_root=root_path)

Found 200 images in datasets_generated_ready_models/stylegan2/generated_images


100%|██████████| 200/200 [00:45<00:00,  4.35it/s]

Finished, saved to datasets_generated_ready_models/stylegan2/classifier_predictions_top_1_cars.csv
Finished, saved to datasets_generated_ready_models/stylegan2/classifier_predictions_top_5_cars.csv


# Object detection

Source: https://docs.ultralytics.com/models/yolov8/

In [6]:
from ultralyticsplus import YOLO

# load model
model = YOLO('ultralyticsplus/yolov8m')

# known classes from https://huggingface.co/ultralyticsplus/yolov8s
known_classes = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

# set model parameters
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.45  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 100  # maximum number of detections per image

In [7]:
from PIL import Image
import os
import zipfile
from tqdm import tqdm
import shutil


def detect_objects_on_images(folder_name, output_suffix, drive_path_root, known_classes=known_classes):

    path_to_images = drive_path_root + '/' + folder_name

    image_paths = [path_to_images +'/'+p for p in os.listdir(path_to_images) if p.endswith(".png")]
    print(f'Found {len(image_paths)} images in {path_to_images}')

    detected_objects = []
    results_images_with_boxes = []

    for image_path in tqdm(image_paths):
        image = Image.open(image_path).convert("RGB")
        results = model.predict(image, verbose=False)

        # add detected objects with confidence to list
        for i in range(len(results)):
            for j in range(len(results[i].boxes)):
                predicted_class = known_classes[results[i].boxes[j].cls.int().item()]
                confidence = round(results[i].boxes[j].conf.item(), 2)

                detected_objects.append({
                    "image_path": image_path,
                    "predicted_class": predicted_class,
                    "confidence": confidence
                })

        # add image with boxes to list
        results_images_with_boxes.append({
            "image_path": image_path,
            "results": results[0]
        })

    output_detected_objects_name = f"detected_objects_{output_suffix}.csv"
    output_images_with_boxes_name = f"images_with_boxes_{output_suffix}"

    # save detected_objects as a csv
    df = pd.DataFrame(detected_objects)
    df.to_csv(output_detected_objects_name, index=True)

    # save images_with_boxes as a zip
    os.makedirs(output_images_with_boxes_name, exist_ok=True)
    for image in results_images_with_boxes:
        image['results'].save(output_images_with_boxes_name + '/' + image['image_path'].split('/')[-1])

    # zip the folder
    shutil.make_archive(output_images_with_boxes_name, 'zip', output_images_with_boxes_name)

    # # copy to google drive
    shutil.copyfile(output_detected_objects_name, drive_path_root + '/' + output_detected_objects_name)
    print("Finished, saved to", drive_path_root + '/' + output_detected_objects_name)

    shutil.copyfile(output_images_with_boxes_name + '.zip', drive_path_root + '/' + output_images_with_boxes_name + '.zip')
    print("Finished, saved to", drive_path_root + '/' + output_images_with_boxes_name + '.zip')
    
    # # clean up
    shutil.rmtree(output_images_with_boxes_name)
    os.remove(output_detected_objects_name)
    os.remove(output_images_with_boxes_name + '.zip')

In [8]:
root_path = "datasets_generated_ready_models/stylegan2"
folder_name = "generated_images"
output_suffix = 'cars'

detect_objects_on_images(folder_name, output_suffix, drive_path_root=root_path)

Found 200 images in datasets_generated_ready_models/stylegan2/generated_images


100%|██████████| 200/200 [00:40<00:00,  4.94it/s]


Finished, saved to datasets_generated_ready_models/stylegan2/detected_objects_cars.csv
Finished, saved to datasets_generated_ready_models/stylegan2/images_with_boxes_cars.zip
